# 정리

- Sub-task Detection
- Model Matching
- Sub-task Execution
- Response Generation

In [3]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')

# Sub-task Detection
유저와 대화하면서 어떤 task를 진행해야하는지 판단하고 대화를 이어나가는 역할

In [4]:
sub_task_list = ["User Preference Eliction", "Recommendation", "Explanation", "Item Information Search"]
sub_task_schema = {
    "User Preference Eliction": {
        "explanation": "Task to elict user preference.",
        "arguments": ["category", "dialogue context"]
    },
    "Recommendation": {
        "explanation": "Task to recommend item.",
        "arguments": ["category", "dialogue context"]
    },
    "Explanation": {
        "explanation": "Task to explain reasons why recommend the item.",
        "arguments": ["category", "dialogue context"]
    },
    "Item Information Search": {
        "explanation": "Task to search information of the item",
        "arguments": ["category", "dialogue context"]
    }
}

In [7]:
llm = ChatOpenAI(api_key=openai_api_key, 
                 model='gpt-4o-mini',
                 temperature=0,
                 max_tokens=2048)

template = """The AI assistant should analyze the user query to detect which sub-task should be selected. 
The subtasks include: {Sub-Task List}. 
The selected sub-task need to represent by its corresponding sub-task schema.
The sub-task schema is {Sub-Task Schema}. 
The sub-task MUST be selected from the above list and represented by the schema.

user query: {user_query}
selected task:
"""

prompt = PromptTemplate(template=template, input_variables=['Sub-Task List', 'Sub-Task Schema', 'user_query'])

chain = prompt | llm

In [8]:
user_query = "I like romance movie"

chain.invoke({"Sub-Task List":"/".join(sub_task_list), 
              "Sub-Task Schema":sub_task_schema,
              "user_query": user_query}
             )

AIMessage(content="{'User Preference Eliction': {'explanation': 'Task to elict user preference.', 'arguments': ['movie', 'I like romance movie']}}", response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 206, 'total_tokens': 237}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-89f714f4-b240-4fa7-9d3f-bc1c5e35d5e9-0')

In [9]:
user_query = "Recommend some movie"

chain.invoke({"Sub-Task List":"/".join(sub_task_list), 
              "Sub-Task Schema":sub_task_schema,
              "user_query": user_query}
             )

AIMessage(content="{'Recommendation': {'explanation': 'Task to recommend item.', 'arguments': ['movie', 'dialogue context']}}", response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 205, 'total_tokens': 230}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-9bb5990b-12fb-4409-8f79-2a2a0fc23a13-0')

In [10]:
user_query = "Why do you recommend that movie?"

chain.invoke({"Sub-Task List":"/".join(sub_task_list), 
              "Sub-Task Schema":sub_task_schema,
              "user_query": user_query}
             )

AIMessage(content="{'Explanation': {'explanation': 'Task to explain reasons why recommend the item.', 'arguments': ['movie', 'dialogue context']}}", response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 208, 'total_tokens': 237}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-475e5f25-8539-4bc7-9d0f-5961503a3734-0')

In [12]:
user_query = "When was the movie made?"

chain.invoke({"Sub-Task List":"/".join(sub_task_list), 
              "Sub-Task Schema":sub_task_schema,
              "user_query": user_query}
             )

AIMessage(content="{'Item Information Search': {'explanation': 'Task to search information of the item', 'arguments': ['movie', 'dialogue context']}}", response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 207, 'total_tokens': 237}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-94ec7050-abd6-4412-8b9d-3b83a47c5b01-0')

In [26]:
template = """The AI assistant should analyze the dialogue context to detect which sub-task should be selected. 
The subtasks include: { Sub-Task List }. 
The selected sub-task need to represent by its corresponding sub-task schema.
The sub-task schema is { Sub-Task Schema }. 
There are some cases for your reference: { Demonstrations }.
The dialogue context is { Dialogue Context }. 
From this dialogue context, which sub-task should be selected?
The sub-task MUST be selected from the above list and represented by the schema"""

prompt = PromptTemplate(template=template, input_variables=['Sub-Task List', 'Sub-Task Schema', 'Demonstrations', 'Dialogue Context'])